# **Tugas 1 (Crawling Data)**



*   Tanggapan masyarakat di media sosial yaitu Twitter yang membahas mengenai Puan Maharani yang akan maju sebagai calon presiden pada periode tahun 2024 - 2029.
*   Portal Tugas Akhir Universitas Trunojoyo Madura.
*   Berita Online.

Crawling data adalah proses mengumpulkan data secara otomatis dari berbagai sumber di internet dengan menggunakan program yang disebut "web crawler" atau "spider". Web crawler adalah program yang dirancang untuk mengunjungi situs web dan mengumpulkan informasi yang berguna, seperti teks, gambar, atau link. Data yang dikumpulkan oleh web crawler kemudian dapat dianalisis dan digunakan untuk berbagai keperluan seperti riset pasar, pemantauan harga, analisis sentimen, dan sebagainya. Namun, penting untuk diingat bahwa pengumpulan data secara otomatis dapat melanggar hak cipta atau privasi, sehingga perlu memperhatikan etika dan hukum dalam penggunaannya.


## Import Library

Library yang dibutuhan antara lain:

1. BeautifulSoup = library untuk mengcrawling data berita
2. snscrape = library untuk mengcrawling data twitter
3. pandas
4. CSV

In [ ]:
!pip3 install snscrape 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 KB 2.8 MB/s eta 0:00:00


Installasi BeautifulSoup

In [ ]:
!pip install requests
!pip install html5lib
!pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

Fungsi untuk Analisis Sentimen

In [ ]:
from textblob import TextBlob
import re
def clean_tweet(tweet):
  return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def analize_sentiment(tweet):
  analysis = TextBlob(clean_tweet(tweet))
  if analysis.sentiment.polarity > 0:
    return 1
  elif analysis.sentiment.polarity == 0:
    return 0
  else:
    return -1

  def analize_subjectorobject(tweet):
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
      return 'Positif'
    elif analysis.sentiment.polarity == 0:
      return 'Neutral'
    else:
      return 'Negatif'
    

## Crawling Data dari Twitter

In [ ]:
tweets = []

In [ ]:
# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('Puan Maharani Calon Presiden since:2022-01-01 until:2023-02-20').get_items()):
    if i>500:
        break
    tweets.append([tweet.date, tweet.user.username, tweet.content, analize_sentiment(tweet.content), tweet.user.location ])

# Creating a dataframe from the tweets list above
tweets_df = pd.DataFrame(tweets, columns=['Datetime', 'Username Twitter', 'Tweet', 'Label', 'Lokasi'])

<ipython-input-6-6ab6758e0693>:5: FutureWarning: content is deprecated, use rawContent instead
  tweets.append([tweet.date, tweet.user.username, tweet.content, analize_sentiment(tweet.content), tweet.user.location ])


In [ ]:
tweets_df['Label'] = tweets_df['Label'].replace([0,1,-1], ['Netral', 'Positif', 'Negatif'])

In [ ]:
tweets_df

,Datetime,Username Twitter,Tweet,Label,Lokasi
0,2023-02-10 07:31:05+00:00,6589O,"Sebagai tokoh perempuan, nama Ketua DPR RI Pua...",Netral,
1,2023-02-10 07:28:59+00:00,6589O,Diusulkan sebagai bakal calon presiden. Puan M...,Netral,
2,2023-02-10 07:28:55+00:00,6589O,Sebanyak 38 DPC bulat mendukung Puan Maharani ...,Netral,
3,2023-02-10 07:28:51+00:00,6589O,Kader PDIP yang layak menjadi calon presiden p...,Netral,
4,2023-02-10 07:28:23+00:00,6589O,Ketua DPR RI Puan Maharani kembali mendapat du...,Netral,
...,...,...,...,...,...
496,2023-01-28 07:33:20+00:00,bemyheal,Ketua DPR Puan Maharani yang masuk bursa calon...,Netral,
497,2023-01-28 07:16:42+00:00,bemyheal,Politikus Gerindra Arief Poyuono menilai PDI P...,Netral,
498,2023-01-28 07:05:46+00:00,bemyheal,"Elektabilitas Ketua DPR RI Puan Maharani, mula...",Netral,
499,2023-01-28 07:03:01+00:00,bemyheal,Puan Maharani mempunyai peluang besar untuk di...,Netral,


In [ ]:
tweets_df.to_csv('puanmaharani.csv', index=False)

## Crawling Data dari Portal Tugas Akhir Universitas Trunojoyo

In [ ]:
import requests
from bs4 import BeautifulSoup

pta = []

for i in range(10,61):
  URL = "https://pta.trunojoyo.ac.id/c_search/byprod/10/" + str(i+1)
  r = requests.get(URL)
  soup = BeautifulSoup(r.text, 'html.parser')
  items = soup.find('div', {'id':'content_journal'})

  for it in items.findAll('li',{'data-cat':'#luxury'}):
    tempNama = it.span.text.strip('\n').replace('Penulis : ', '')
    # links = it.findAll('div', {'style':'margin: 25px 0px 15px 0px;'})
    link = it.find('a', 'gray button')['href']
    req = requests.get(link)
    soup2 = BeautifulSoup(req.text, 'html.parser')
    abstraks = soup2.findAll('div', {'style':'margin: 15px 15px 15px 15px;'})
    abstrak = soup2.find('p', {'align':'justify'}).text
  
    tempNIM = link.replace('https://pta.trunojoyo.ac.id/welcome/detail/', '')

    pta.append([it.a.text.strip('\n'), tempNIM, tempNama, abstrak])
    
pta_df = pd.DataFrame(pta, columns=['Judul', 'NIM', 'Penulis', 'Abstrak'])
pta_df

,Judul,NIM,Penulis,Abstrak
0,IMPLEMENTASI METODE N-GRAM PADA PEMBUATAN GAM...,080411100078,Farizal Arifin,Teknologi mobile game berkembang dengan sa...
1,IMPLEMENTASI PERFORMANCE PRISM \r\nUNTUK SISTE...,070411100081,Luhur Priyantoko Aji,Pengukuran kinerja adalah proses di mana organ...
2,PENGEMBANGAN VIRTUAL CLASS UNTUK PEMBELAJARAN ...,070411100042,Dwi Adi Permana,Internet berkembang dengan cepat beberapa tahu...
3,ANALISIS DAN PENERAPAN ALGORITMA PARTICLE SWAR...,070411100129,Qomarul Haryadi Irfan Rifa'i,Penjadwalan mata kuliah di program studi Tekni...
4,Rancang Bangun Sistem Pendukung Keputusan Pene...,080411100003,Ayu Winda Istara,Seiring dengan perubahan dalam paradigma pemba...
...,...,...,...,...
250,Pemetaan SMA/SMK/MA Di Kabupaten Lamongan Berd...,100411100054,Durorin Khumairoh,"Sarana dan prasarana yang memadai, jumlah lulu..."
251,REKOMENDASI PEMINJAMAN BUKU BERDASARKAN MINAT ...,100411100051,NAILY NAJIHAH,"Banyaknya keragaman buku di perpustakaan, meny..."
252,âVirtual Environment(VE) Kampus menggunakan ...,100411100026,Yusuf Machsar,Kampus merupakan lingkungan pendidikan perguru...
253,SISTEM PENDUKUNG KEPUTUSAN PENENTUAN PEMINATAN...,100411100079,Atin Hasanah,ABSTRAK \r\n\r\nTahun Ajaran 2013/2014 akan di...


In [ ]:
pta_df.to_csv('ptatrunojoyo.csv', index=False)

## Crawling Data Berita Online (Detik.com)

In [ ]:
import requests as req
from bs4 import BeautifulSoup as bs
from datetime import datetime
import csv

userAgent = {'user-agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Mobile Safari/537.36'}


a = 1
for page in range(0, 30):
  url = f'https://www.detik.com/search/searchall?query=politik&siteid=2&sortby=time&page={page}'
  ge = req.get(url,userAgent).text
  sop = bs(ge, 'lxml')
  li = sop.find('div', class_='list media_rows list-berita')
  lin = li.find_all('article')
  for x in lin:
    link = x.find('a')['href']
    headline = x.find('a').find('h2').text
    ge_ = req.get(link,userAgent).text
    sop_ = bs(ge_,'lxml')
    content = sop_.find_all('div', class_='detail__body-text itp_bodycontent')
    for x in content:
      x = x.find_all('p')
      y = [y.text for y in x]
      content_ = ''.join(y).replace('\n','').replace('ADVERTISMENT','').replace('SCROLL TO RESUME CONTENT', '')
      print(f'done[{a}] >> {headline[0:]}')
      a += 1
      with open('beritaonline.csv', 'a') as file:
        wr = csv.writer(file, delimiter=',')
        wr.writerow([headline,content_])

done[1] >> Terkuak! Ini Dia Sosok Kejutan yang Diusulkan Jokowi Jadi Gubernur BI
done[2] >> NasDem Tak Masalah Anies Merosot di Survei Litbang Kompas: Jadi Pelecut
done[3] >> RK Jadi Capres Alternatif Populer, Pengamat: Sinyal untuk Golkar
done[4] >> Putin Tuding Barat soal Ukraina, Biden Bilang Tak Ada yang Serang Rusia
done[5] >> Kajati DKI Dorong Adanya Kajian Pembentukan UU Restorative Justice
done[6] >> Gus Muhaimin Ingin Kebudayaan Jadi Panglima Pembangunan Bangsa
done[7] >> AHY Jajal Mobil Esemka, Begini Komentar Gun Romli
done[8] >> Sederet Tugas Berat Gubernur Bank Indonesia, Apa Saja?
done[9] >> Perubahan Sistem Pemilu Dinilai Harus Sejalan dengan Amanah Konstitusi
done[10] >> Terkuak! Ini Dia Sosok Kejutan yang Diusulkan Jokowi Jadi Gubernur BI
done[11] >> NasDem Tak Masalah Anies Merosot di Survei Litbang Kompas: Jadi Pelecut
done[12] >> RK Jadi Capres Alternatif Populer, Pengamat: Sinyal untuk Golkar
done[13] >> Putin Tuding Barat soal Ukraina, Biden Bilang Tak Ada yang Se